# 03 — Report Modeling Tabular (Keras/TF)

In [3]:
from pathlib import Path
import json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

import optuna
from optuna.storages import JournalStorage, JournalFileStorage
from optuna.storages.journal._file import JournalFileOpenLock


## Config

In [4]:
ROOT = Path.cwd() # Path(__file__).resolve().parent
DATA_CLEAN = (ROOT / "../data/clean/base_dataset.csv").resolve()
OUT_DIR    = (ROOT / "../outputs").resolve()
ART_DIR    = (OUT_DIR / "artifacts_keras").resolve()
FIG_DIR    = (OUT_DIR / "figures").resolve()
FIG_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
JOURNAL_PATH = (OUT_DIR / "optuna_tabular_keras.journal").resolve()
LOCK = JournalFileOpenLock(str(JOURNAL_PATH) + ".lock")
STORAGE = JournalStorage(JournalFileStorage(str(JOURNAL_PATH), lock_obj=LOCK))

TARGET_COL = "GHI"
DEFAULT_INPUT_STEPS = 36
DEFAULT_HORIZON_STEPS = 6

## Utils

In [6]:
def build_lstm(L, n_feat, units=64, layers_n=1, dropout=0.0, bidir=False):
    inp = layers.Input(shape=(L, n_feat))
    x = inp
    for _ in range(layers_n-1):
        cell = layers.LSTM(units, return_sequences=True, dropout=dropout)
        if bidir: cell = layers.Bidirectional(cell)
        x = cell(x)
    cell = layers.LSTM(units, dropout=dropout)
    if bidir: cell = layers.Bidirectional(cell)
    x = cell(x)
    out = layers.Dense(1, dtype="float32")(x)
    return models.Model(inp, out)

def build_gru(L, n_feat, units=64, layers_n=1, dropout=0.0, bidir=False):
    inp = layers.Input(shape=(L, n_feat))
    x = inp
    for _ in range(layers_n-1):
        cell = layers.GRU(units, return_sequences=True, dropout=dropout)
        if bidir: cell = layers.Bidirectional(cell)
        x = cell(x)
    cell = layers.GRU(units, dropout=dropout)
    if bidir: cell = layers.Bidirectional(cell)
    x = cell(x)
    out = layers.Dense(1, dtype="float32")(x)
    return models.Model(inp, out)

def build_dilated_like(L, n_feat, units=64, dilation=2, dropout=0.0):
    inp = layers.Input(shape=(L, n_feat))
    x = layers.Lambda(lambda t: t[:, ::dilation, :])(inp)
    x = layers.LSTM(units, dropout=dropout)(x)
    out = layers.Dense(1, dtype="float32")(x)
    return models.Model(inp, out)

def build_clockwork(L, n_feat, hidden=60, modules=3, base_period=1, dropout=0.0):
    assert hidden % modules == 0
    h_per = hidden // modules
    periods = [base_period * (2**m) for m in range(modules)]
    inp = layers.Input(shape=(L, n_feat))
    h_list = []
    for p in periods:
        xt = layers.Lambda(lambda t, step=p: t[:, ::step, :])(inp)
        ht = layers.SimpleRNN(h_per, activation="tanh", dropout=dropout)(xt)
        h_list.append(ht)
    h = layers.Concatenate()(h_list) if len(h_list) > 1 else h_list[0]
    out = layers.Dense(1, dtype="float32")(h)
    return models.Model(inp, out)

def build_seq_arrays(X_2d, y_1d, L, horizon):
    N, F = X_2d.shape
    outX, outy = [], []
    last = N - L - horizon + 1
    for i in range(max(0,last)):
        outX.append(X_2d[i:i+L])
        outy.append(y_1d[i + L + horizon - 1])
    return np.asarray(outX, dtype="float32"), np.asarray(outy, dtype="float32")

def metrics_from_scaled(pred_scaled, true_scaled, y_scaler):
    p = y_scaler.inverse_transform(pred_scaled.reshape(-1,1)).ravel()
    t = y_scaler.inverse_transform(true_scaled.reshape(-1,1)).ravel()
    mae = mean_absolute_error(t, p)
    rmse = float(np.sqrt(mean_squared_error(t, p)))
    mape = float(np.mean(np.abs((t + 1e-6) - p) / (np.abs(t) + 1e-6)) * 100)
    smape = float(100 * np.mean(2*np.abs(p - t) / (np.abs(t) + np.abs(p) + 1e-6)))
    r2 = float(r2_score(t, p))
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape, "sMAPE": smape, "R2": r2}, (t, p)

def plot_series(y_true, y_pred, title, n=1000, fname=None):
    plt.rcParams["axes.grid"] = True
    plt.rcParams["grid.alpha"] = 0.25
    n = min(n, len(y_true))
    fig, ax = plt.subplots(figsize=(11,3.8))
    ax.plot(y_true[:n], label="Real", lw=1.2)
    ax.plot(y_pred[:n], label="Pred", lw=1.0, alpha=0.95)
    ax.set_title(title); ax.set_xlabel("Time steps (10-min)"); ax.set_ylabel("GHI (W/m²)")
    ax.legend(frameon=False); fig.tight_layout()
    if fname: fig.savefig(fname, dpi=140)
    plt.close(fig)

## Data

In [7]:
df = pd.read_csv(DATA_CLEAN, parse_dates=[0], index_col=0).sort_index()
df.index.name = "time"
base_feats = [
    'Presion','TempAmb','WindSpeed','WindDirection',
    'hour_sin','hour_cos','DoY Sin','DoY Cos',
    'solar_zenith','solar_azimuth','solar_elevation',
    'TempAmb_roll1h_mean','TempAmb_roll6h_mean',
    'Presion_roll1h_mean','Presion_roll6h_mean',
    'WindSpeed_roll1h_mean','WindSpeed_roll6h_mean',
    'temp_pressure_ratio','wind_temp_interaction'
]
ghi_lags  = [c for c in ['GHI_lag1','GHI_lag3','GHI_lag6','GHI_lag12','GHI_lag36'] if c in df.columns]
ghi_rolls = [c for c in ['GHI_roll1h_mean','GHI_roll3h_mean','GHI_roll6h_mean','GHI_roll1h_max'] if c in df.columns]
feat_cols = [c for c in base_feats if c in df.columns] + ghi_lags + ghi_rolls

n = len(df); i_tr = int(0.7*n); i_va = int(0.85*n)
df_train, df_val, df_test = df.iloc[:i_tr], df.iloc[i_tr:i_va], df.iloc[i_va:]
X_scaler = StandardScaler(); y_scaler = StandardScaler()
X_train = X_scaler.fit_transform(df_train[feat_cols].values)
X_val   = X_scaler.transform(df_val[feat_cols].values)
X_test  = X_scaler.transform(df_test[feat_cols].values)
y_train = y_scaler.fit_transform(df_train[[TARGET_COL]].values).ravel()
y_val   = y_scaler.transform(df_val[[TARGET_COL]].values).ravel()
y_test  = y_scaler.transform(df_test[[TARGET_COL]].values).ravel()
imp = SimpleImputer(strategy="median")
X_train = imp.fit_transform(X_train); X_val = imp.transform(X_val); X_test = imp.transform(X_test)


## Reload/Rebuild

In [8]:
# ---------------- Reload best trials from journal ----------------
def load_best(name):  # returns optuna.study.Study
    return optuna.create_study(study_name=name, storage=STORAGE, load_if_exists=True, direction="minimize")

study_rf = load_best("RF_RMSE")
study_lstm = load_best("LSTM_MSEval")
study_gru = load_best("GRU_MSEval")
study_dil = load_best("DilatedRNN_MSEval")
study_cw = load_best("ClockworkRNN_MSEval")

# ---------------- Rebuild models, load weights, predict ----------------
# RF (no pesos externos)
best_rf = RandomForestRegressor(random_state=42, n_jobs=-1, **study_rf.best_trial.params)
best_rf.fit(np.vstack([X_train, X_val]), np.concatenate([y_train, y_val]))
rf_metrics, (y_true_rf, y_pred_rf) = metrics_from_scaled(best_rf.predict(X_test), y_test, y_scaler)
plot_series(y_true_rf, y_pred_rf, "RandomForest Optuna — Test (sample)", fname=FIG_DIR / "pred_rf_opt_sample.png")

# Redes
def rebuild_from_trial(study, arch):
    p = study.best_trial.params
    L = study.best_trial.user_attrs.get("seq_len_used", p.get("input_steps", DEFAULT_INPUT_STEPS))
    H = study.best_trial.user_attrs.get("horizon_used",  p.get("horizon_steps", DEFAULT_HORIZON_STEPS))
    Xtr_seq, ytr_seq = build_seq_arrays(np.vstack([X_train, X_val]),
                                        np.concatenate([y_train, y_val]), L, H)
    Xte_seq, yte_seq = build_seq_arrays(X_test, y_test, L, H)
    n_feat = Xtr_seq.shape[2]
    if arch == "LSTM":
        model = build_lstm(L, n_feat, p.get("hidden",64), p.get("num_layers",1), p.get("dropout",0.0))
        ck = ART_DIR / "best_lstm.weights.h5"
    elif arch == "GRU":
        model = build_gru(L, n_feat, p.get("hidden",64), p.get("num_layers",1), p.get("dropout",0.0))
        ck = ART_DIR / "best_gru.weights.h5"
    elif arch == "DILATED":
        model = build_dilated_like(L, n_feat, p.get("hidden",64), p.get("dilation",2), p.get("dropout",0.0))
        ck = ART_DIR / "best_dilated.weights.h5"
    else:
        model = build_clockwork(L, n_feat, p.get("hidden",120), p.get("modules",3), p.get("base_period",1), p.get("dropout",0.0))
        ck = ART_DIR / "best_clockwork.weights.h5"
    model.load_weights(str(ck))
    yhat = model.predict(Xte_seq, verbose=0).squeeze()
    y_true = y_scaler.inverse_transform(yte_seq.reshape(-1,1)).ravel()
    y_pred = y_scaler.inverse_transform(yhat.reshape(-1,1)).ravel()
    return y_true, y_pred

ytrue_lstm, ypred_lstm = rebuild_from_trial(study_lstm, "LSTM")
plot_series(ytrue_lstm, ypred_lstm, "LSTM — Test (sample)", fname=FIG_DIR / "pred_lstm_sample.png")

ytrue_gru, ypred_gru = rebuild_from_trial(study_gru, "GRU")
plot_series(ytrue_gru, ypred_gru, "GRU — Test (sample)", fname=FIG_DIR / "pred_gru_sample.png")

ytrue_dil, ypred_dil = rebuild_from_trial(study_dil, "DILATED")
plot_series(ytrue_dil, ypred_dil, "Dilated — Test (sample)", fname=FIG_DIR / "pred_dilated_sample.png")

ytrue_cw, ypred_cw = rebuild_from_trial(study_cw, "CLOCKWORK")
plot_series(ytrue_cw, ypred_cw, "Clockwork — Test (sample)", fname=FIG_DIR / "pred_clockwork_sample.png")

# Linear & Persistence
lin = LinearRegression().fit(np.vstack([X_train, X_val]), np.concatenate([y_train, y_val]))
lin_metrics, (ytrue_lin, ypred_lin) = metrics_from_scaled(lin.predict(X_test), y_test, y_scaler)
plot_series(ytrue_lin, ypred_lin, "LinearRegression — Test (sample)", fname=FIG_DIR / "pred_linear_sample.png")

def persistence_baseline(y_scaled, horizon):
    y_hat = np.roll(y_scaled, horizon)
    y_hat[:horizon] = y_scaled[horizon]
    return y_hat

y_pers = persistence_baseline(y_test, DEFAULT_HORIZON_STEPS)
pers_metrics, (ytrue_pers, ypred_pers) = metrics_from_scaled(y_pers, y_test, y_scaler)
plot_series(ytrue_pers, ypred_pers, "Persistence — Test (sample)", fname=FIG_DIR / "pred_persistence_sample.png")

# (Puedes copiar aquí el resto de funciones del bloque 1 para scatter, residuales, overlay y errores por hora/bin)
print("Report figures saved to:", FIG_DIR)

[I 2025-10-14 17:03:01,516] Using an existing study with name 'RF_RMSE' instead of creating a new one.
[I 2025-10-14 17:03:01,524] Using an existing study with name 'LSTM_MSEval' instead of creating a new one.
[I 2025-10-14 17:03:01,532] Using an existing study with name 'GRU_MSEval' instead of creating a new one.
[I 2025-10-14 17:03:01,542] Using an existing study with name 'DilatedRNN_MSEval' instead of creating a new one.
[I 2025-10-14 17:03:01,550] Using an existing study with name 'ClockworkRNN_MSEval' instead of creating a new one.
I0000 00:00:1760479431.155572  890921 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22136 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1760479432.061029  891132 cuda_dnn.cc:529] Loaded cuDNN version 90101
I0000 00:00:1760479439.272130  891132 service.cc:148] XLA service 0x7ff0117a1d40 initialized for platform CUDA (this does not guarantee th

Report figures saved to: /mnt/SOLARLAB/E_Ladino/Repo_2/solar-forecasting-colombia/outputs/figures
